In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM,Embedding
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

import numpy as np
import random
import sys
import os
import collections
import re
import pandas as pd

Using TensorFlow backend.


In [2]:
rnn_size = 300 # size of RNN
batch_size = 30 # minibatch size
seq_length = 5 # sequence length
num_epochs = 16 # number of epochs
#learning_rate = 0.001 #learning rate
sequences_step = 1 #step to create sequences

In [3]:
data = pd.read_json("5campaign.json")
data

,favorite_count,in_reply_to_user_id_str,is_retweet,retweet_count,text,month,year,hour,week_year,date,modded_text,modded1,noTags_punct,modded1_stemmed,labels
18431,33,NaN,False,12,"""@JaayZen: @realDonaldTrump Thanks Donald. Now...",1,2015,0,1,2015-01-01,"""@jaayzen: @realdonaldtrump thanks donald. now...",jaayzen realdonaldtrump thanks donald run pres...,jaayzen realdonaldtrump thanks donald now run ...,jaayzen realdonaldtrump thank donald run ...,0
18432,28,NaN,False,9,"""@teamconquerca: First time in the""Trump Inter...",1,2015,0,1,2015-01-01,"""@teamconquerca: first time in the""trump inter...",teamconquerca first time thetrump internationa...,teamconquerca first time in thetrump internati...,teamconquerca first time thetrump intern ...,3
18433,36,NaN,False,15,"""@AngelaTN777: @gochico @realDonaldTrump: He S...",1,2015,0,1,2015-01-01,"""@angelatn777: @gochico @realdonaldtrump: he s...",angelatn777 realdonaldtrump stands believes re...,angelatn777 realdonaldtrump he stands up for...,angelatn777 realdonaldtrump stand believ ...,2
18434,18,NaN,False,11,"""@JaayZen: @grandmiapens He has all the ingred...",1,2015,0,1,2015-01-01,"""@jaayzen: @grandmiapens he has all the ingred...",jaayzen ingredients get elected turn ship arou...,jaayzen he has all the ingredients to get el...,jaayzen ingredi get elect turn ship aro...,3
18435,17,NaN,False,7,"""@MarcAardvark: @realDonaldTrump @ronmeier123 ...",1,2015,0,1,2015-01-01,"""@marcaardvark: @realdonaldtrump @ronmeier123 ...",marcaardvark realdonaldtrump ronmeier123 one b...,marcaardvark realdonaldtrump ronmeier123 one o...,marcaardvark realdonaldtrump ronmeier123 o...,2
18436,26,NaN,False,7,"""@PaulParmar1: @realDonaldTrump so excited for...",1,2015,0,1,2015-01-01,"""@paulparmar1: @realdonaldtrump so excited for...",paulparmar1 realdonaldtrump excited apprentice...,paulparmar1 realdonaldtrump so excited for app...,paulparmar1 realdonaldtrump excit apprenti...,1
18437,47,NaN,False,17,"""@ryanbushby: @realDonaldTrump i bet you have ...",1,2015,0,1,2015-01-01,"""@ryanbushby: @realdonaldtrump i bet you have ...",ryanbushby realdonaldtrump bet great ideas tur...,ryanbushby realdonaldtrump i bet you have some...,ryanbushbi realdonaldtrump bet great idea...,1
18438,45,NaN,False,19,"""@LLAnkers: DonaldTrump, it's a shame the lef...",1,2015,0,1,2015-01-01,"""@llankers: donaldtrump, it's a shame the left...",llankers donaldtrump shame left try hard tear ...,llankers donaldtrump its a shame the left try ...,llanker donaldtrump shame left tri hard ...,4
18439,58,NaN,False,18,"""@flicka__: @realDonaldTrump for president""",1,2015,6,1,2015-01-01,"""@flicka__: @realdonaldtrump for president""",flicka__ realdonaldtrump president,flicka__ realdonaldtrump for president,flicka__ realdonaldtrump presid,2
18440,77,NaN,False,27,The Mar-a-Lago Club was amazing tonight. Every...,1,2015,7,1,2015-01-01,the mar-a-lago club was amazing tonight. every...,maralago club amazing tonight everybody bigges...,the maralago club was amazing tonight everybod...,maralago club amaz tonight everybodi big...,3


In [4]:
t = []
for idx,row in data.iterrows():
    l = re.findall(r'(?<!\w)(@\w{1,15})\b',row["text"])
    g = row["text"]
    for el in l:
        g = g.replace(el," ")
    t.append(g)
data["t"] = t


In [5]:
#remove links
data["t"] = data["t"].str.replace("http\S+"," ")
data["t"] = data["t"].str.replace("\n", " ")

#remove punctuation
data['t'] = data["t"].str.replace("[^\w\s]"," ")
data["t"] = data["t"].str.lower()

sentences = data[data["labels"] == 5]["t"].tolist()
sentences1 = data[data["labels"]== 1]["t"].tolist()
sentences2 = data[data["labels"]== 0]["t"].tolist()
sentences3 = data[data["labels"]== 6]["t"].tolist()

sentences = sentences + sentences1 #+sentences2 # +sentences3
#sentences = sentences[0:2500]
len(sentences)

2138

In [6]:
data["t"]

18431          thanks donald  now run for president  fu...
18432        first time in the trump international hote...
18433             he stands up for what he believes in ...
18434          he has all the ingredients to get electe...
18435            one of the best things i bought in 201...
18436          so excited for   thank you for creating ...
18437          i bet you have some great ideas on how t...
18438        donaldtrump   it s a shame the left try so...
18439                                       for president 
18440    the mar a lago club was amazing tonight  every...
18441             for president of the united states   ...
18442          you re only a year away from  2016    pi...
18443                         do u ever sleep    not much 
18444          happy new year sir can t wait for  celeb...
18445          so excited for a new year  amp  new cele...
18446          i believe in you to bring this country b...
18447                             for president please  

In [7]:
fullText = " ".join(sentences)
text_in_words = [w for w in fullText.split(' ')]

text = list(map(lambda s: s.strip(""), text_in_words))
text1 = []
for el in text:
    if el != "":
        text1.append(el)
text = text1

word_counts = collections.Counter(text1)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common() if x[0] != '\n' and x[0] != " "]
vocabulary_inv = list(sorted(vocabulary_inv))

vocabulary_inv

['00',
 '000',
 '00pm',
 '1',
 '10',
 '100',
 '1000',
 '10th',
 '11',
 '12',
 '1237',
 '12k',
 '13',
 '1314',
 '13m',
 '14',
 '14th',
 '15',
 '16',
 '16th',
 '17',
 '18',
 '19',
 '1k',
 '1st',
 '2',
 '20',
 '2001',
 '2004',
 '2013',
 '2015',
 '2016',
 '2016gopnomination',
 '2017',
 '20k',
 '223',
 '226th',
 '233',
 '24',
 '241st',
 '25',
 '25k',
 '25th',
 '26',
 '27',
 '28',
 '2a',
 '2m',
 '2nite',
 '3',
 '30',
 '30pm',
 '32',
 '33',
 '34',
 '35',
 '360',
 '38',
 '39',
 '3b',
 '3pm',
 '3rd',
 '4',
 '40',
 '400',
 '42',
 '43',
 '44',
 '450',
 '47',
 '49',
 '4th',
 '5',
 '500',
 '50k',
 '51',
 '5th',
 '6',
 '61',
 '610',
 '62',
 '64',
 '6days',
 '7',
 '72',
 '727',
 '74',
 '756',
 '757',
 '770',
 '7pm',
 '7pme',
 '8',
 '800',
 '8000',
 '81',
 '823',
 '858',
 '8pm',
 '8th',
 '9',
 '900',
 '917',
 '99',
 'a',
 'abc',
 'aberdeen',
 'ability',
 'abingdon',
 'able',
 'about',
 'absolute',
 'absolutely',
 'abt',
 'ac360',
 'acceptance',
 'accepting',
 'access',
 'accolade',
 'accomplish',
 'ac

In [8]:
# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]


#size of the vocabulary
vocab_size = len(words)

#create sequences
sequences = []
next_words = []
for i in range(0, len(text) - seq_length, sequences_step):
    if text[i] != "" and text[i+seq_length] != "":
        sequences.append(text[i: i + seq_length])
        next_words.append(text[i + seq_length])

print('nb sequences:', len(sequences))
print(sequences[0],sequences[1],sequences[2])


nb sequences: 28488
['so', 'excited', 'for', 'thank', 'you'] ['excited', 'for', 'thank', 'you', 'for'] ['for', 'thank', 'you', 'for', 'creating']


In [9]:
print('Vectorization.')
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1


Vectorization.


In [10]:
filepath="val_weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [11]:
# build the model: a single LSTM
print('Build LSTM model.')
model = Sequential()
#model.add(Embedding(vocab_size,300, input_length=seq_length))
model.add(LSTM(rnn_size)) #input_shape=(seq_length, vocab_size)))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))

#adam optimizer
optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=["accuracy"])

#fit the model
model.fit(X, y,batch_size=batch_size,epochs=num_epochs,verbose = 1, validation_split=0.30,callbacks=callbacks_list)

Build LSTM model.
Train on 19941 samples, validate on 8547 samples
Epoch 1/16
19941/19941 [==============================] - 89s 4ms/step - loss: 6.3593 - acc: 0.0814 - val_loss: 6.4230 - val_acc: 0.0831

Epoch 00001: val_acc improved from -inf to 0.08307, saving model to val_weights-improvement-01-0.08.hdf5
Epoch 2/16
19941/19941 [==============================] - 89s 4ms/step - loss: 5.8712 - acc: 0.0901 - val_loss: 6.3245 - val_acc: 0.1196

Epoch 00002: val_acc improved from 0.08307 to 0.11957, saving model to val_weights-improvement-02-0.09.hdf5
Epoch 3/16
19941/19941 [==============================] - 89s 4ms/step - loss: 5.4218 - acc: 0.1391 - val_loss: 6.1636 - val_acc: 0.1575

Epoch 00003: val_acc improved from 0.11957 to 0.15748, saving model to val_weights-improvement-03-0.14.hdf5
Epoch 4/16
19941/19941 [==============================] - 88s 4ms/step - loss: 4.8610 - acc: 0.1903 - val_loss: 6.2520 - val_acc: 0.1684

Epoch 00004: val_acc improved from 0.15748 to 0.16836, savin

In [12]:
test = ["obama is a bad president","crooked hillary","Mexico will pay the wall",
        "The best president","Donald Trump is"
       , "China is working against","Make america great again","Obamacare is failing","my campaign is",
        "vote me because hillary is","Melania"
       ,  "cnn should stop publishing ", "Today in ", "hillary should","Watch my debate", "Stupid","democrats know",
        "nytimes is the worst","biased media","phony"
       ]

In [15]:
model.load_weights("val_weights-improvement-04-0.19.hdf5")


In [21]:
words_number = 15 #number of words to generate
seed_sentences = "iowa" #sentence for seed generation

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#initiate sentences
generated = ''
sentence = []
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)
print('Generating text with the following seed: "' + ' '.join(sentence) + '"')

print ()

#generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 1)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

print(generated)

print()


Generating text with the following seed: "a a a a iowa"

a a a a iowa go urgency salesmen greatly am to heads robert a estate states us we take change

